In [2]:
library(rstan)

Loading required package: ggplot2
Loading required package: StanHeaders
rstan (Version 2.16.2, packaged: 2017-07-03 09:24:58 UTC, GitRev: 2e1f913d3ca3)
For execution on a local, multicore CPU with excess RAM we recommend calling
rstan_options(auto_write = TRUE)
options(mc.cores = parallel::detectCores())


In [3]:
sessionInfo()

R version 3.3.2 (2016-10-31)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: OS X El Capitan 10.11.6

locale:
[1] ja_JP.UTF-8/ja_JP.UTF-8/ja_JP.UTF-8/C/ja_JP.UTF-8/ja_JP.UTF-8

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] rstan_2.16.2         StanHeaders_2.16.0-1 ggplot2_2.2.1       

loaded via a namespace (and not attached):
 [1] Rcpp_0.12.8         magrittr_1.5        munsell_0.4.3      
 [4] uuid_0.1-2          colorspace_1.3-2    R6_2.2.0           
 [7] stringr_1.2.0       plyr_1.8.4          tools_3.3.2        
[10] grid_3.3.2          gtable_0.2.0        lazyeval_0.2.0     
[13] digest_0.6.12       assertthat_0.1      tibble_1.2         
[16] crayon_1.3.2        gridExtra_2.2.1     IRdisplay_0.4.4    
[19] repr_0.12.0         IRkernel_0.8.6.9000 inline_0.3.14      
[22] evaluate_0.10       pbdZMQ_0.2-5        stringi_1.1.5      
[25] scales_0.4.1        stats4_3.3.2        jsonlite_1.

In [30]:
# データ
N <- 10
x_samples <- 1:N
y_samples <- 1:N
for(i in 1:N){
    x <- round(runif(1, min=3, max=7), 1) # 3.0〜7.0までの乱数
    mu <- exp(1.5+0.1*x)
    y <- rpois(1, lambda=mu)
    x_samples[i] <- x
    y_samples[i] <- y
}
print(x_samples)
print(y_samples)

 [1] 3.7 3.1 6.4 5.1 5.1 3.6 6.3 5.4 6.5 3.3
 [1] 13  9  3 10  6  7  5 13  8  7


In [33]:
stan_data <- list(N=N, x=x_samples, y=y_samples)
model <- "
data {
    int<lower=1>N;
    vector[N] x;
    int<lower=0> y[N];
}
parameters {
    real beta1;
    real beta2;
}
model {
    for (i in 1:N){
        y[i] ~ poisson(exp(beta1+beta2*x[i]));
    }
    beta1 ~ normal(0, 1000);
    beta2 ~ normal(0, 1000);
}
"
fit <- stan(model_code=model, data=stan_data, iter=1000, chains=1)
fit


SAMPLING FOR MODEL 'b84d10f4dc37f511239d8adcc932a241' NOW (CHAIN 1).

Gradient evaluation took 9e-06 seconds
1000 transitions using 10 leapfrog steps per transition would take 0.09 seconds.
Adjust your expectations accordingly!


Iteration:   1 / 1000 [  0%]  (Warmup)
Iteration: 100 / 1000 [ 10%]  (Warmup)
Iteration: 200 / 1000 [ 20%]  (Warmup)
Iteration: 300 / 1000 [ 30%]  (Warmup)
Iteration: 400 / 1000 [ 40%]  (Warmup)
Iteration: 500 / 1000 [ 50%]  (Warmup)
Iteration: 501 / 1000 [ 50%]  (Sampling)
Iteration: 600 / 1000 [ 60%]  (Sampling)
Iteration: 700 / 1000 [ 70%]  (Sampling)
Iteration: 800 / 1000 [ 80%]  (Sampling)
Iteration: 900 / 1000 [ 90%]  (Sampling)
Iteration: 1000 / 1000 [100%]  (Sampling)

 Elapsed Time: 0.033398 seconds (Warm-up)
               0.025105 seconds (Sampling)
               0.058503 seconds (Total)



Inference for Stan model: b84d10f4dc37f511239d8adcc932a241.
1 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=500.

       mean se_mean   sd  2.5%   25%   50%   75% 97.5% n_eff Rhat
beta1  2.66    0.04 0.41  1.96  2.40  2.63  2.92  3.53    96 1.01
beta2 -0.12    0.01 0.09 -0.31 -0.17 -0.11 -0.06  0.03    98 1.01
lp__  88.17    0.09 0.99 85.48 87.76 88.45 88.94 89.17   110 1.00

Samples were drawn using NUTS(diag_e) at Sun Aug 13 16:31:52 2017.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).